<span style="font-size: 200%; color: red;">First! check the changing "your config.yaml" </span>

## [seq] import

In [1]:
import os
import json
import yaml
import sys
import time
import copy
import IPython.display as ipd
import pprint
from pathlib import Path
from tqdm import tqdm


import numpy as np
import torch
import torchaudio
from librosa.filters import mel as librosa_mel_fn
#import matplotlib
#matplotlib.use("Agg")
import matplotlib.pyplot as plt
from scipy.io.wavfile import write


import toybox

In [2]:
def plot_audio(audio, samplerate, title='time-domain waveform'):
    """
    usage:
        # audio is [channel, time(num_frames)] ex.torch.Size([1, 68608])
        # audio[0,:]: list of 1ch audio data
        # audio.shape[1]: int value of 1ch audio data length
        audio, sample_rate = torchaudio.load(str(iwav_path))
        %matplotlib inline
        plot_audio(audio, sample_rate)
    """
    # transform to mono
    channel = 0
    audio = audio[channel,:].view(1,-1)
    # to numpy
    audio = audio.to('cpu').detach().numpy().copy()
    time = np.linspace(0., audio.shape[1]/samplerate, audio.shape[1])
    
    fig, ax = plt.subplots(figsize=(12,9))
    
    ax.plot(time, audio[0, :])
    ax.set_title(title, fontsize=20, y=-0.12)
    ax.tick_params(direction='in')
    #ax.set_xlim(0, 3)
    ax.set_xlabel('Time')
    ax.set_ylabel('Amp')
    #ax.legend()
    plt.tight_layout()
    fig.canvas.draw()
    plt.show()
    #fig.savefig('figure.png')
    plt.close(fig)
    return fig

def plot_mel(tensors:list, titles:list[str]):
    """
    usage:
        mel = mel_process(...)
        fig_mel = plot_mel([mel_groundtruth[0], mel_prediction[0]],
                            ['groundtruth', 'inferenced(model)'])

    """
    xlim = max([t.shape[1] for t in tensors])
    fig, axs = plt.subplots(nrows=len(tensors),
                            ncols=1,
                            figsize=(12, 9),
                            constrained_layout=True)

    if len(tensors) == 1:
        axs = [axs]
    
    for i in range(len(tensors)):
        im = axs[i].imshow(tensors[i],
                           aspect="auto",
                           origin="lower",
                           interpolation='none')
        #plt.colorbar(im, ax=axs[i])
        fig.colorbar(im, ax=axs[i])
        axs[i].set_title(titles[i])
        axs[i].set_xlim([0, xlim])
    fig.canvas.draw()
    #plt.show()
    #plt.close()
    plt.close(fig)  # fig.close() 
    return fig

def convert_phn_to_id(phonemes, phn2id):
    """
    phonemes: phonemes separated by ' '
    phn2id: phn2id dict
    """
    return [phn2id[x] for x in ['<bos>'] + phonemes.split(' ') + ['<eos>']]


def text2phnid(text, phn2id, language='en', add_blank=True):
    if language == 'en':
        from text import G2pEn
        word2phn = G2pEn()
        phonemes = word2phn(text)
        if add_blank:
            phonemes = ' <blank> '.join(phonemes)
        return phonemes, convert_phn_to_id(phonemes, phn2id)
    else:
        raise ValueError(
            'Language should be en (for English)!')

In [3]:
#ckpt_file_dir: logs4model/<model_name>/<runtime_name>/ckpt/
#run_tfkfulmask_k3,run_tfkfulplus_k3 by gradtfkfultts
model_name = 'gradtfk5tts' #  gradseptts, gradtfktts, gradtfk5tts, gradtimektts, gradfreqktts, gradtfkful_mask, gradtfkful_plus
model_dir = 'run_tfk_k5'
runtime_name = 'infer4colb'
hifigan_versions = 'LJ_V1'
config_path4model = 'configs/config_exp_mid.yaml'
wav_dir_name = f'wav_{hifigan_versions}'
test_ds_path = Path('configs/test_dataset.json') #Path(config['test_datalist_path'])
#~/aoi/logs4model/gradtfk5tts/run_tfk_k5/ckpt/gradtfk5tts_500_397001.pt
ckpt_filename= 'gradtfk5tts_500_397001.pt'
# for runtime to load model
print(f'runtime_name: {runtime_name}')
ckpt_dir = f'logs4model/{model_name}/{model_dir}/ckpt'
ckpt_path = Path(ckpt_dir + "/" + ckpt_filename)


if ckpt_path.exists()==True:
    print(f"ckpt_path: {ckpt_path}")
else:
    print(f"Not find")

config = toybox.load_yaml_and_expand_var('configs/config_exp_mid.yaml')
print(model_name)
print(runtime_name)
n_mels: int = config['n_mels'] # 80
n_fft: int = config['n_fft'] # 1024
sample_rate: int = config['sample_rate'] # 22050
hop_size: int = config['hop_size'] # 256
win_size: int = config['win_size'] # 1024
f_min: int = config['f_min'] # 0
f_max: int = config['f_max'] # 8000
random_seed: int = 1234#config['random_seed'] # 42 or 1234
print(n_mels, n_fft, sample_rate, hop_size, win_size, f_min, f_max, random_seed)

print(f"phn2id_path: {config['phn2id_path']}")
with open(config['phn2id_path']) as f:
    phn2id = json.load(f)

vocab_size = len(phn2id) + 1

toybox.set_seed(random_seed)

runtime_name: infer4colb
ckpt_path: logs4model/gradtfk5tts/run_tfk_k5/ckpt/gradtfk5tts_500_397001.pt
gradtfk5tts
infer4colb
80 1024 22050 256 1024 0 8000 1234
phn2id_path: ./configs/phn2id.json


In [4]:
#devicesetting
import os

print(f"all cpu at using device: {os.cpu_count()}")
print(f"Number of available CPU: {len(os.sched_getaffinity(0))}") # Number of available CPUs can also be obtained. ,use systemcall at linux.
#print(f"GPU_name: {torch.cuda.get_device_name()}\nGPU avail: {torch.cuda.is_available()}\n")

DEVICE = 'cpu' #'cuda' or 'cpu'
#DEVICE_HiFiGAN = 'cuda'
#if torch.cuda.is_available():
#    print('use cuda')
#else:
#    os._exit(os.EX_OK)
#    print('use cpu')

#device = torch.device(DEVICE_HiFiGAN)
device = torch.device(DEVICE)
print(f'device: {device}')

all cpu at using device: 52
Number of available CPU: 4
device: cpu


In [5]:
RESULT_DIR_PATH = Path(f'./result4eval/{runtime_name}/{model_name}/{DEVICE}/e500_n50')
RESULT_MEL_DIR_PATH = RESULT_DIR_PATH / 'mel_LJ_V1'
RESULT_WAV_DIR_PATH = RESULT_DIR_PATH / wav_dir_name
#RESULT_JSON_PATH = RESULT_DIR_PATH / f'eval4mid_{hifigan_versions}.json'
#eval_jsonl_path = RESULT_DIR_PATH / 'eval4mid.json'
RESULT_JSON_PATH = RESULT_DIR_PATH / f'eval4mid_{hifigan_versions}.json'

print(RESULT_DIR_PATH)
print(RESULT_MEL_DIR_PATH)
print(RESULT_WAV_DIR_PATH)

result4eval/infer4colb/gradtfk5tts/cpu/e500_n50
result4eval/infer4colb/gradtfk5tts/cpu/e500_n50/mel_LJ_V1
result4eval/infer4colb/gradtfk5tts/cpu/e500_n50/wav_LJ_V1


In [6]:
# check path

# for text2mel
print('test_ds_path-----------------------------------------')
if test_ds_path.exists():
    print(f'Exists {str(test_ds_path)}')
    with open(config['test_datalist_path']) as j:
        test_ds_list = json.load(j)
    print(f'loaded {test_ds_path}')
else:
    print(f'No exist {test_ds_path}')

print('RESULT_DIR_PATH-------------------------------------------')
if RESULT_DIR_PATH.exists():
    print(f'Exists {RESULT_DIR_PATH}')
else:
    RESULT_DIR_PATH.mkdir(parents=True)
    print(f'No exist {RESULT_DIR_PATH}')

print('RESULT_MEL_DIR_PATH-------------------------------------------')
if RESULT_MEL_DIR_PATH.exists():
    print(f'Exists {RESULT_MEL_DIR_PATH}')
else:
    RESULT_MEL_DIR_PATH.mkdir(parents=True)
    print(f'No exist {RESULT_MEL_DIR_PATH}')

print('RESULT_WAV_DIR_PATH-------------------------------------------')
if RESULT_WAV_DIR_PATH.exists():
    print(f'Exists {RESULT_WAV_DIR_PATH}')
else:
    RESULT_WAV_DIR_PATH.mkdir(parents=True)
    print(f'No exist {RESULT_WAV_DIR_PATH}')

print('RESULT_JSON_PATH-------------------------------------------')
if RESULT_JSON_PATH.exists():
    print(f'Exists {RESULT_JSON_PATH}')
else:
    #RESULT_DIR_PATH.mkdir(parents=True)
    print(f'No exist {RESULT_JSON_PATH}')

test_ds_path-----------------------------------------
Exists configs/test_dataset.json
loaded configs/test_dataset.json
RESULT_DIR_PATH-------------------------------------------
Exists result4eval/infer4colb/gradtfk5tts/cpu/e500_n50
RESULT_MEL_DIR_PATH-------------------------------------------
No exist result4eval/infer4colb/gradtfk5tts/cpu/e500_n50/mel_LJ_V1
RESULT_WAV_DIR_PATH-------------------------------------------
No exist result4eval/infer4colb/gradtfk5tts/cpu/e500_n50/wav_LJ_V1
RESULT_JSON_PATH-------------------------------------------
Exists result4eval/infer4colb/gradtfk5tts/cpu/e500_n50/eval4mid_LJ_V1.json


## [seq] load model 

In [7]:
# import models
from gradtts import GradTTS
from gradseptts import GradSepTTS
from gradtfktts import GradTFKTTS
from gradtfk5tts import GradTFKTTS as GradTFK5TTS
from gradtimektts import GradTimeKTTS
from gradfreqktts import GradFreqKTTS
from gradtfkfultts import GradTFKFULTTS

print(model_name)
print("[seq] loading Model")

print("loading diffusion-TTS ===================================")
N_STEP = 50
TEMP = 1.5

print('loading ', ckpt_path)
_, _, state_dict = torch.load(ckpt_path,
                            map_location=device)

#with open(config_path4model) as f:
#    config = yaml.load(f, yaml.SafeLoader)
config4model = toybox.load_yaml_and_expand_var(config_path4model)

print("[seq] Initializing diffusion-TTS...")
if model_name == "gradtts":
    model = GradTTS.build_model(config4model, vocab_size)
elif model_name == "gradseptts":
    model = GradSepTTS.build_model(config4model, vocab_size)
elif model_name == "gradtfktts":
    model = GradTFKTTS.build_model(config4model, vocab_size)
elif model_name == "gradtfk5tts":
    model = GradTFK5TTS.build_model(config4model, vocab_size)
elif model_name == "gradtfkfultts":
    model = GradTFKFULTTS.build_model(config4model, vocab_size)
elif model_name == "gradtimektts":
    model = GradTimeKTTS.build_model(config4model, vocab_size)
elif model_name == "gradfreqktts":
    model = GradFreqKTTS.build_model(config4model, vocab_size)
else:
    raise ValueError(f"Error: '{model_name}' is not supported")

model = model.to(device)
model.load_state_dict(state_dict)
print(f'Number of encoder + duration predictor parameters: {model.encoder.nparams/1e6}m')
print(f'Number of decoder parameters: {model.decoder.nparams/1e6}m')
print(f'Total parameters: {model.nparams/1e6}m')

print("loading HiFi-GAN ===================================")
#setting file paths
# from https://github.com/huawei-noah/Speech-Backbones/tree/main/Grad-TTS/hifi-gan
# https://drive.google.com/drive/folders/1-eEYTB5Av9jNql0WGBlRoi-WH2J7bp5Y?usp=sharing
if hifigan_versions == 'LJ_V1':
    HiFiGAN_CONFIG = f'./hifigan/official_pretrained/LJ_V1/config.json'
    HiFiGAN_ckpt = './hifigan/official_pretrained/LJ_V1/generator_v1'
elif hifigan_versions == 'LJ_V2':
    HiFiGAN_CONFIG = './hifigan/official_pretrained/LJ_V2/config.json'
    HiFiGAN_ckpt = './hifigan/official_pretrained/LJ_V2/generator_v2'
else:
    print('Dont supported.')


from hifigan import models, env

with open(HiFiGAN_CONFIG) as f:
    hifigan_hparams = env.AttrDict(json.load(f))

hifigan_randomseed = hifigan_hparams.seed
print(f'hifigan_randomseed: {hifigan_randomseed}')

# generator ===================
print("[seq] loading HiFiGAN")
vocoder = models.Generator(hifigan_hparams)

vocoder.load_state_dict(torch.load(
    HiFiGAN_ckpt, map_location=device)['generator'])
vocoder = vocoder.eval().to(device)
vocoder.remove_weight_norm()

print("loading UTMOS ===================================")
predictor_utmos = torch.hub.load("tarepan/SpeechMOS:v1.2.0", "utmos22_strong", trust_repo=True)

gradtfk5tts
[seq] loading Model
loading diffusion-TTS ===================================
loading  logs4model/gradtfk5tts/run_tfk_k5/ckpt/gradtfk5tts_500_397001.pt
[seq] Initializing diffusion-TTS...
Number of encoder + duration predictor parameters: 3.549137m
Number of decoder parameters: 2.044639m
Total parameters: 5.593776m
loading HiFi-GAN ===================================
hifigan_randomseed: 1234
[seq] loading HiFiGAN


/work/sora-sa/aoi/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Removing weight norm...
loading UTMOS ===================================


Using cache found in /work/sora-sa/.cache/torch/hub/tarepan_SpeechMOS_v1.2.0


In [8]:
infer_data_num: int = 101 #len(test_ds_list) is 200
print(f'infer_data_num: {infer_data_num}')
print(f'RESULT_DIR_PATH: {RESULT_DIR_PATH}')
print(f'RESULT_MEL_DIR_PATH: {RESULT_DIR_PATH}')
print(f'RESULT_WAV_DIR_PATH: {RESULT_WAV_DIR_PATH}')
print(f'RESULT_JSON_PATH: {RESULT_JSON_PATH}')

infer_data_num: 101
RESULT_DIR_PATH: result4eval/infer4colb/gradtfk5tts/cpu/e500_n50
RESULT_MEL_DIR_PATH: result4eval/infer4colb/gradtfk5tts/cpu/e500_n50
RESULT_WAV_DIR_PATH: result4eval/infer4colb/gradtfk5tts/cpu/e500_n50/wav_LJ_V1
RESULT_JSON_PATH: result4eval/infer4colb/gradtfk5tts/cpu/e500_n50/eval4mid_LJ_V1.json


## [seq] infer

In [9]:
eval_list = []


for i in tqdm(range(infer_data_num)):
    test_ds_filename = test_ds_list[i]['name']
    mel_npy_path = RESULT_MEL_DIR_PATH / f"{test_ds_filename}.npy"
    synth_wav_path = RESULT_WAV_DIR_PATH / f"{test_ds_filename}.wav"
    print(f'test_ds_index_{i}: {test_ds_filename}')
    # [seq]text2mel =========================================================
    # load txt
    print('[seq]text2mel')
    text = test_ds_list[i]['text']
    phonemes, phnid = text2phnid(text, phn2id, 'en')
    phonemes_len_int = len(phonemes)
    phnid_len_int = len(phnid)
    print(f'phonemes_len: {phonemes_len_int}')
    print(f'phnid_len: {phnid_len_int}')
    phnid_len = torch.tensor(len(phnid), dtype=torch.long).unsqueeze(0).to(device)
    phnid = torch.tensor(phnid).unsqueeze(0).to(device)

    # [seq] synth speech
    # process text to mel
    # mel is [n_mels, n_frame]
    start_time = time.perf_counter()
    _, mel_prediction, _ = model.forward(phnid,
                                        phnid_len,
                                        n_timesteps=N_STEP,
                                        temperature=TEMP,
                                        solver='original')
    end_time = time.perf_counter()

    dt = end_time - start_time
    dt4mel = dt * 22050 / ( mel_prediction.shape[-1] * 256)
    print(f'{model_name} dt: {dt}')
    print(f'{model_name} RTF: {dt4mel}')
    
    # for save mel
    mel4save = mel_prediction.unsqueeze(0) # [batch, channel(freq), n_frame(time)] ex.[1, 80, 619]
    # save
    #mel_npy_path =  RESULT_MEL_DIR_PATH / f"{test_ds_filename}.npy"
    #print(f'test_ds_index_{i}: {mel_npy_path}')
    np.save(mel_npy_path, mel4save.cpu().detach().numpy().copy())

    # [seq]mel2wav =========================================================
    print('[seq]mel2wav')
    x = np.load(mel_npy_path) # [1, n_mel, n_frame]
    x2audio = torch.FloatTensor(x).to(device)
    x2audio = x2audio.squeeze().unsqueeze(0)
    # x2audio is [1, n_mels, n_frames]
    assert x2audio.shape[0] == 1
    with torch.no_grad():
        # vocoder.forward(x).cpu() is torch.Size([1, 1, 167168])
        audio = (vocoder.forward(x2audio).cpu().squeeze().clamp(-1,1).numpy() * 32768).astype(np.int16)
    write(
        synth_wav_path,
        hifigan_hparams.sampling_rate,
        audio)

    # [seq]wav2utmos =========================================================
    print('[seq]wav2utmos')
    #iwav_path = RESULT_WAV_DIR_PATH / f"{filename}.wav"
    #wav, samplerate = torchaudio.load(iwav_path)
    wav, samplerate = torchaudio.load(synth_wav_path)
    score_utmos = predictor_utmos(wav, samplerate)
    score_utmos_float = score_utmos.item()
    print(f'utmos: {score_utmos_float}')
    #eval_dict = {'name': filename, 'path': str(iwav_path), 'utmos': score_float}
    #score_utmos_list.append(eval_dict)
    
    # path, テキスト文、phonimes, phonimes数, dt, RTF, utmos
    eval_dict = {
        'name': test_ds_filename,
        'wav_path': synth_wav_path
        'phonemes_len': phonemes_len_int,
        'phnid_len': phnid_len_int,
        'dt': dt,
        'RTF4mel': dt4mel,
        'utmos': score_utmos_float
    }
    eval_list.append(eval_dict)
    

  0%|                                                                                                                                                          | 0/101 [00:00<?, ?it/s]

test_ds_index_0: LJ045-0049
[seq]text2mel
phonemes_len: 1127
phnid_len: 215
gradtfk5tts dt: 18.204049011692405
gradtfk5tts RTF: 2.165698812520597
[seq]mel2wav
[seq]wav2utmos


  1%|█▍                                                                                                                                                | 1/101 [00:26<44:32, 26.72s/it]

utmos: 4.020130634307861
test_ds_index_1: LJ017-0027
[seq]text2mel
phonemes_len: 411
phnid_len: 79
gradtfk5tts dt: 6.0498573537915945
gradtfk5tts RTF: 1.981335471885466
[seq]mel2wav
[seq]wav2utmos


  2%|██▉                                                                                                                                               | 2/101 [00:35<26:38, 16.15s/it]

...

utmos: 4.085937976837158
test_ds_index_99: LJ029-0081
[seq]text2mel
phonemes_len: 1427
phnid_len: 269
gradtfk5tts dt: 21.1776631642133
gradtfk5tts RTF: 1.9040623074231116
[seq]mel2wav
[seq]wav2utmos


 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 100/101 [25:30<00:20, 20.24s/it]

utmos: 4.281447410583496
test_ds_index_100: LJ017-0230
[seq]text2mel
phonemes_len: 1085
phnid_len: 207
gradtfk5tts dt: 13.309624452143908
gradtfk5tts RTF: 1.6377076962598949
[seq]mel2wav
[seq]wav2utmos


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 101/101 [25:48<00:00, 15.34s/it]

utmos: 3.5967650413513184


In [10]:
#RESULT_JSON_PATH = RESULT_DIR_PATH / 'eval4mid.json'
if RESULT_JSON_PATH.exists() == False:
    with open(RESULT_JSON_PATH, 'w') as f:
        for entry in eval_list:
            f.write(json.dumps(entry) + '\n')
    print(f'Make {RESULT_JSON_PATH}')
else:
    print(f'Already Exists {RESULT_JSON_PATH}')

Already Exists result4eval/infer4colb/gradtfk5tts/cpu/e500_n50/eval4mid_LJ_V1.json


In [ ]:
# recheck

In [11]:
a = toybox.load_json('result4eval/infer4colb/gradtfk5tts/cpu/e500_n50/eval4mid_LJ_V1.json')

Exist result4eval/infer4colb/gradtfk5tts/cpu/e500_n50/eval4mid_LJ_V1.json


In [12]:
dt_list = [a[n]['dt'] for n in range(len(a))]
RTF4mel_list = [a[n]['RTF4mel'] for n in range(len(a))]
utmos_list = [a[n]['utmos'] for n in range(len(a))]

dt_nparr = np.array(dt_list[1:101])
RTF4mel_nparr = np.array(RTF4mel_list[1:101])
utmos_nparr = np.array(utmos_list[1:101])
print(len(dt_nparr))

significant_digits = 8

# for culc difference time to infer text2mel
dt_mean = toybox.round_significant_digits(np.mean(dt_nparr), significant_digits=significant_digits)
dt_var = toybox.round_significant_digits(np.var(dt_nparr), significant_digits=significant_digits)
dt_std = toybox.round_significant_digits(np.std(dt_nparr), significant_digits=significant_digits)
print(f'dt ---------------------------')
print(f'dt mean: {dt_mean}')
print(f'dt var: {dt_var}')
print(f'dt std: {dt_std}')

# for culc RTF4mel to infer text2mel
RTF4mel_mean = toybox.round_significant_digits(np.mean(RTF4mel_nparr), significant_digits=significant_digits)
RTF4mel_var = toybox.round_significant_digits(np.var(RTF4mel_nparr), significant_digits=significant_digits)
RTF4mel_std = toybox.round_significant_digits(np.std(RTF4mel_nparr), significant_digits=significant_digits)
print(f'RTF ---------------------------')
print(f'RTF mean: {RTF4mel_mean}')
print(f'RTF var: {RTF4mel_var}')
print(f'RTF std: {RTF4mel_std}')

# for culc utmos to infer 
print(f'utmos ---------------------------')
utmos_mean = toybox.round_significant_digits(np.mean(utmos_nparr), significant_digits=significant_digits)
utmos_var = toybox.round_significant_digits(np.var(utmos_nparr), significant_digits=significant_digits)
utmos_std = toybox.round_significant_digits(np.std(utmos_nparr), significant_digits=significant_digits)
print(f'utmos mean: {utmos_mean}')
print(f'utmos var: {utmos_var}')
print(f'utmos std: {utmos_std}')

100
dt ---------------------------
dt mean: 10.763684
dt var: 13.17348
dt std: 3.629529
RTF ---------------------------
RTF mean: 1.6718893000000001
RTF var: 0.0087506937
RTF std: 0.093545142
utmos ---------------------------
utmos mean: 4.1353975
utmos var: 0.089310245
utmos std: 0.2988482
